# Loggning

In [ ]:
import logging

logger = logging.getLogger(__name__)

# logging.basicConfig(filename='test_log.txt', level=logging.DEBUG)
logging.basicConfig(
    filename='test_log.txt', 
    format='[%(asctime)s][%(levelname)s] %(message)s', 
    level=logging.WARNING)

In [ ]:
logger.setLevel(logging.CRITICAL)

In [ ]:
logger.debug('Debug')
logger.info('Hello')
logger.warning('Look out!')
logger.critical('Time for a break!')

In [ ]:
'20240904'[6:]

In [ ]:
import datetime
import logging
import pandas as pd

logging.basicConfig(
    filename='DataCleaner_log.txt', 
    format='[%(asctime)s][%(levelname)s] %(message)s', 
    level=logging.DEBUG)

class DataCleaner:
    def __init__(self, data) -> None:
        self.data = data
        self.logger = logging.getLogger(__name__)

    def clean_dates(self) -> None:
        for row in self.data:
            if '-' in row['date']:
                try:
                    y, m, d = [int(x) for x in row['date'].split('-')]
                except Exception as e:
                    self.logger.error((e, row['date']))
                    new_date = pd.NaT
                else:
                    new_date = datetime.date(y, m, d)
            elif '/' in row['date']:
                try:
                    m, d, y = [int(x) for x in row['date'].split('/')]
                except Exception as e:
                    self.logger.error((e, row['date']))
                    new_date = pd.NaT
                else:
                    new_date = datetime.date(2000 + y, m, d)
            # Hantera YYYYMMDD/YYMMDD-format!
            elif row['date'].isdigit():
                if len(row['date']) == 8:
                    y = row['date'][:4]
                    m = row['date'][4:6]
                    d = row['date'][6:]
                    y, m, d = [int(x) for x in [y, m, d]]
                    new_date = datetime.date(y, m, d)
                elif len(row['date']) == 6:
                    y = row['date'][:2]
                    m = row['date'][2:4]
                    d = row['date'][4:]
                    y, m, d = [int(x) for x in [y, m, d]]
                    new_date = datetime.date(y, m, d)
                else:
                    self.logger.error('Unsupported number of digits for date %s', row['date'])
                    new_date = pd.NaT
            else:
                self.logger.error('Could not parse date %s', row['date'])
                new_date = pd.NaT
            row['date'] = new_date

    def clean_temps(self):
        for row in self.data:
            try:
                new_temp = float(row['temp'])
            except TypeError:  
                new_temp = pd.NA
            except ValueError:  
                new_temp = float(row['temp'].replace(',', '.'))
            except Exception as e:
                self.logger.error((e, row['temp']))
                new_temp = pd.NA

            row['temp'] = new_temp

    def clean(self):
        self.clean_dates()
        self.clean_temps()
        return self.data


In [ ]:
import json

data = json.load(open('../lektion_3/data/temps3.json'))

dc = DataCleaner(data=data)
cleaned_data = dc.clean()

In [ ]:
cleaned_data